In [1]:
# Set date of reassignment
REASSIGNMENT_DATE = "2025_10_22" # Date of running this code, in YYYY_MM_DD format

In [2]:
# Set up
import pandas as pd
import numpy as np
import sys
from pathlib import Path
CODE_ROOT = Path.cwd().parents[1]
sys.path.append(str(CODE_ROOT))
import config
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
import os
from lab_assignment import assign_enumerators

In [6]:
assignments = pd.read_csv(config.ENUMERATORS / f"reassignedlabs_{REASSIGNMENT_DATE}.csv")
all_enumerators = pd.read_csv(config.ENUMERATORS / "all_enumerators.csv")

# Merge to get enum_foldername
assignments = assignments.merge(all_enumerators[['enum_id', 'foldername']], on='enum_id', how='left')

In [8]:
# Create reassignments file for each enumerator

# Rename columns for clarity
assignments = assignments.rename(columns={"Professor": "Lab Responsible Person"})
assignments = assignments.rename(columns={"Source": "Website"})

# Columns to include in the enumerator's file
cols_to_include = [
    "labgroupid", "Lab Group", "Faculty", "Institute", 
    "Lab Responsible Person", "Email", "Website", "Treatment Status"
]

# Color treatment yellow and control no color
fill_colors = {
    "treatment": "FFFF00",  # Yellow
    "control": "FFFFFF"     # No color (white)
}

for enum_id, enum_data in assignments.groupby("enum_id"):

    # Get enumerator info
    id = enum_data["enum_id"].iloc[0]
    name = enum_data["foldername"].iloc[0]
    folder_name = f"{name}_data"

    # Columns to include
    labs_for_enum = enum_data[cols_to_include].copy()

    # Create excel path
    filename = os.path.join(config.SWITCHDRIVE_ROOT, folder_name, f"new_lab_assignment_{REASSIGNMENT_DATE}.xlsx")

    # Save first without formatting
    labs_for_enum.to_excel(filename, index=False)

    # Load workbook and select active sheet
    wb = load_workbook(filename)
    ws = wb.active
    ws.title = "Lab Assignments"

    # Bold header row
    for cell in ws[1]:
        cell.font = Font(bold=True)

    # Adjust column widths
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column name
        for cell in col:
            try:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    # Fill colors based on treatment status
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=ws.min_column, max_col=ws.max_column):
        status_cell = row[cols_to_include.index("Treatment Status")]
        status = str(status_cell.value).lower()
        if status in fill_colors:
            status_cell.fill = PatternFill(start_color=fill_colors[status], end_color=fill_colors[status], fill_type="solid")
    
    # Save workbook
    wb.save(filename)